In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import ogr, os
import shapely
import geopandas
import pandas
import gdal
import copy
from shapely.geometry import MultiLineString, LineString, MultiPoint
import numpy as np
import json
from pathlib import Path

import fiona
import pytess

In [ ]:
#In this first section we convert out polygons into lines by taking the boundary of the building polygons

expol = geopandas.read_file("Polygons/buildings.shp") #<---CHANGE TO OWN DIRECTORY
x = expol.geometry.boundary
expol.geometry = x
fig = pyplot.figure(figsize=(12,12))

ax1 = fig.add_subplot(221)
ax1.set_title("linestring")
expol.plot(ax=ax1, facecolor='Red')

expol = expol[['osm_id', 'geometry']]

#Make shape file
out=Path("Polygons\polyboundary.shp") # <---CHANGE THIS TO OWN DIRECTORY
expol.to_file(out)

In [ ]:
#In this block we are converting the boundary lines we made above into points

bpol = geopandas.read_file("Polygons/polyboundary.shp") #<---CHANGE TO OWN DIRECTORY

lpdata=geopandas.GeoDataFrame()
lpdata['geometry']=None
lpdata['bound']=None

p=0
for i in range(len(bpol)):
    line_string=bpol.geometry[i]
    for k in np.arange(0.1, 0.9, 0.05):
        ip=line_string.interpolate(k, normalized=True)
        lpdata.loc[p, 'geometry']=ip
        lpdata.loc[p, 'bound']=bpol['osm_id'][i]
        p+=1

out=Path("Polygons\polypoints.shp") #<---CHANGE TO OWN DIRECTORY
lpdata.to_file(out) 

lpdata.plot(markersize=0.2)

In [ ]:
#Here we are doing the same thing as above, but with street data which is already in a linestring
#We need to convert this into points

streets=geopandas.read_file('lines/streets.shp') #<---CHANGE TO OWN DIRECTORY

lpdata1=geopandas.GeoDataFrame()
lpdata1['geometry']=None
lpdata1['street']=None

p=0
for i in range(len(streets)):
    line_string=streets.geometry[i]
    for k in np.arange(0.1, 0.9, 0.5):
        ip=line_string.interpolate(k, normalized=True)
        lpdata1.loc[p, 'geometry']=ip
        lpdata1.loc[p, 'street']=streets['osm_id'][i]
        p+=1
        
out=Path("Lines\linepointstreet.shp") #<---CHANGE TO OWN DIRECTORY
lpdata1.to_file(out)

lpdata1.plot(markersize=0.2)

In [ ]:
#Here we have three point files, buildings, streets and bus stops
#We want to merge each shapefile together to create one file with all points within it

pnt = geopandas.read_file("Points/bus_stops.shp") #<---CHANGE TO OWN DIRECTORY
lpnt = geopandas.read_file("Lines/linepointstreet.shp") #<---CHANGE TO OWN DIRECTORY
ppnt = geopandas.read_file("Polygons/polypoints.shp") #<---CHANGE TO OWN DIRECTORY

pnt.head()
lpnt.head()
ppnt.head()

ppnt['id']=ppnt['bound']
lpnt['id']=lpnt['street']
pnt['id']=pnt['osm_id']

pnt.head()

allpnt = pandas.concat([ppnt, lpnt, pnt], sort=True, ignore_index=True)
allpnt.crs = pnt.crs
allpnt = allpnt.to_crs({'init': 'epsg:2193'})

allpnt.plot(markersize=.1)

In [ ]:
#Now we can make our voronoi polygons from the variable allpnt

allpnt['x'] = allpnt['geometry'].x
allpnt['y'] = allpnt['geometry'].y

points_array=np.array([[allpnt['x'][k], allpnt['y'][k]] for k in range(len(allpnt))])

allpnt.tail()

voronois = pytess.voronoi(points_array)

vorpolys = []
for pt, poly in voronois:
    vorpolys.append(shapely.geometry.Polygon(poly))

In [ ]:
#Lastly, we need to dissolve the data by the ID field that we created earlier
#This will join each individual building and street voronoi polygons back to their full shape

poly_df = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(vorpolys))
poly_df.crs = allpnt.crs

poly_df.geometry.head()

voronois_df=geopandas.sjoin(poly_df, allpnt, how='left', op='contains')

voronois_df.head()

voronois_df['geometry'] = voronois_df.buffer(0.01) #create small buffer, because polygons intersect

voronois_dissolv=voronois_df.dissolve(by='id')

voronois_dissolv.head()

out=Path("VoronoisDissolved.shp") #CHANGE TO OWN DIRECTORY
voronois_dissolv.to_file(out)

In [ ]:
#Now we can plot our voronoi polygons!
voronois_dissolv.plot(markersize=.1, facecolor="darkgreen")
plt.xlim(1748500, 1750000)
plt.ylim(5426500, 5427500)

voronois_dissolv.plot(markersize=.1, facecolor="darkgreen")
plt.xlim(1748800, 1749400)
plt.ylim(5426800, 5427200)